In [1]:
%run Main.ipynb

# Variation of the fine-structure constant

## - Models
**Bekenstein’s class of varying $\alpha$ models $\to$ flat $\Lambda$CDM case**

$\frac{\Delta \alpha}{\alpha}(z) = 2 \zeta_m \log(1+z) + \frac{2(\zeta_{\Lambda} - 2\zeta_m)}{3\sqrt{\Omega_{\Lambda}}} \left[ \log\left( \frac{1+\sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_m}}\right) - \sqrt{E^2(z)} \log\left( \frac{\sqrt{\Omega_{\Lambda}} + \sqrt{E^2(z)}}{\sqrt{\Omega_m (1+z)^3}} \right) \right] \Leftrightarrow$ 

$\Leftrightarrow \frac{\Delta \alpha}{\alpha}(z) = \left[ \frac{2}{3\sqrt{\Omega_{\Lambda}}} \left[ \log\left( \frac{1+\sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_m}}\right) - \sqrt{E^2(z)} \log\left( \frac{\sqrt{\Omega_{\Lambda}} + \sqrt{E^2(z)}}{\sqrt{\Omega_m (1+z)^3}} \right) \right] \right] \zeta_{\Lambda} + \left[ 2\log(1+z) - \frac{4}{3\sqrt{\Omega_{\Lambda}}} \left[ \log\left( \frac{1+\sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_m}}\right) - \sqrt{E^2(z)} \log\left( \frac{\sqrt{\Omega_{\Lambda}} + \sqrt{E^2(z)}}{\sqrt{\Omega_m (1+z)^3}} \right) \right] \right] \zeta_m$

For a flat universe we have: $\Omega_m + \Omega_{\Lambda} = 1 \Rightarrow \Omega_{\Lambda} = 1 - \Omega_m$

In [2]:
def zeta_lambda(z, OmegaM, w0, wa):
    if w0 == None: 
        w0 = -1.
    if wa == None: 
        wa = 0
    #in flat ΛCDM case we always have w0=-1 and wa=0
    zeta_lambda = 2/(3*np.sqrt(1-OmegaM)) * (np.log((1+np.sqrt(1-OmegaM))/np.sqrt(OmegaM)) - 
     (function_E_z(z, OmegaM, w0, wa)*np.log((np.sqrt(1-OmegaM) + 
        function_E_z(z, OmegaM, w0, wa))/(np.sqrt(OmegaM * (1+z)**3)))))
    return zeta_lambda

def zeta_m(z, OmegaM, w0, wa):
    if w0 == None: 
        w0 = -1.
    if wa == None: 
        wa = 0
    #in flat ΛCDM case we always have w0=-1 and wa=0
    zeta_m = 2*np.log(1+z) - (4/(3*np.sqrt(1-OmegaM))) * (np.log((1+np.sqrt(1-OmegaM))/np.sqrt(OmegaM)) - 
     (function_E_z(z, OmegaM, w0, wa)*np.log((np.sqrt(1-OmegaM) + 
        function_E_z(z, OmegaM, w0, wa))/(np.sqrt(OmegaM * (1+z)**3)))))
    return zeta_m

def alpha_v_bekenstein(z, OmegaM, w0, wa, Zl, Zm):
    return zeta_lambda(z, OmegaM, w0, wa) * Zl + zeta_m(z, OmegaM, w0, wa) * Zm

**CPL Parametrization**

$w_{CPL}(z) = w_0 + w_a \frac{z}{1+z} $ and $\Omega_{CPL}(z) = \frac{1-\Omega_m}{1-\Omega_m+\Omega_m (1+z)^{-3(w_0+w_a)} \exp(3w_az/(1+z))}$

For $w_\phi\geq-1$:

$\frac{\Delta \alpha}{\alpha}(z) = \zeta \int_{0}^{z} \sqrt{3\Omega_\phi(z') [1+w_\phi(z')]} \,\frac{dz\prime}{1+z\prime}$

For Phantom Energy Fields $(w_\phi<-1)$:

$\frac{\Delta \alpha}{\alpha}(z) = -\zeta \int_{0}^{z} \sqrt{3\Omega_\phi(z') \mid 1+w_\phi(z')\mid} \,\frac{dz'}{1+z'}$

In [3]:
def w_CPL(z, w0, wa):
    w_CPL = w0 + wa * (z/(1+z))
    return w_CPL

def Omega_CPL(z, OmegaM, w0, wa):
    Omega_CPL = (1 - OmegaM) / (1 - OmegaM + (OmegaM*(1+z)**(-3*(w0+wa)) * np.exp((3*wa*z)/(1+z))))
    return Omega_CPL


def function(z, OmegaM, w0, wa):
    if w0 >= -1:            
        return np.sqrt(3*Omega_CPL(z, OmegaM, w0, wa) * (1 + w_CPL(z, w0, wa)))/(1+z)
    if 3*Omega_CPL(z, OmegaM, w0, wa) * (1 + w_CPL(z, w0, wa)) < 0:
        return -np.sqrt(3*Omega_CPL(z, OmegaM, w0, wa) * abs(1 + w_CPL(z, w0, wa)))/(1+z)


def alpha_v_CPL(z, OmegaM, w0, wa, zeta):
    alpha_v_CPL, error = scipy.integrate.quad(function, 0, z, args=(OmegaM, w0, wa),epsabs=10**(-30))
    return zeta * alpha_v_CPL  

## - Derivatives calculation
**Bekenstein**

In [4]:
def dfdOmegaM_ab(z, OmegaM, w0, wa, Zl, Zm):
    return ( alpha_v_bekenstein(z, OmegaM+hh, w0, wa, Zl, Zm)-alpha_v_bekenstein(z, OmegaM, w0, wa, Zl, Zm))/hh

def dfdZl(z, OmegaM, w0, wa):
    return zeta_lambda(z, OmegaM, w0, wa)

def dfdZm(z, OmegaM, w0, wa):
    return zeta_m(z, OmegaM, w0, wa)

**CPL Parametrization**

In [5]:
def dfdOmegaM_a(z, OmegaM, w0, wa, zeta):
    return ( alpha_v_CPL(z, OmegaM+hh, w0, wa, zeta)-alpha_v_CPL(z, OmegaM, w0, wa, zeta) )/hh

def dfdw0_a(z, OmegaM, w0, wa, zeta):
    return ( alpha_v_CPL(z, OmegaM, w0+hh, wa, zeta)-alpha_v_CPL(z, OmegaM, w0, wa, zeta) )/hh

def dfdwa_a(z, OmegaM, w0, wa, zeta):
    return ( alpha_v_CPL(z, OmegaM, w0, wa+hh, zeta)-alpha_v_CPL(z, OmegaM, w0, wa, zeta) )/hh

def dfdzeta(z, OmegaM, w0, wa, zeta):
    return alpha_v_CPL(z, OmegaM, w0, wa, zeta)/zeta

## - Add Priors (Separate)

#### Atomic Clocks

- Considering $\Omega_m$ as a constant:

Present-day drift rate constrained by laboratory tests using atomi clocks $(\Omega_m = 0.3)$:

$\left( \frac{1}{H}\frac{\dot{\alpha}}{\alpha} \right)_0 = - 0.13 \zeta_{\Lambda} - 1.74 \zeta_{m}$

- Considering $\Omega_m$ as a parameter:

Present-day drift rate:

${\left( \frac{1}{H} \frac{\dot{\alpha}}{\alpha} \right)}_0 = - \zeta_{\Lambda} + 2(\zeta_{\Lambda} -2 \zeta_{m}) \frac{\Omega_m}{\sqrt{\Omega_{\Lambda}}} \log{\frac{1 + \sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_{m}}}}\Leftrightarrow$

$\Leftrightarrow {\left( \frac{1}{H} \frac{\dot{\alpha}}{\alpha} \right)}_0 = \left( 2 \frac{\Omega_m}{\sqrt{\Omega_{\Lambda}}} \log{\frac{1 + \sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_{m}}}} - 1\right)\zeta_{\Lambda} - \left(4 \frac{\Omega_m}{\sqrt{\Omega_{\Lambda}}} \log{\frac{1 + \sqrt{\Omega_{\Lambda}}}{\sqrt{\Omega_{m}}}}\right)  \zeta_{m}$

For a flat universe we have: $\Omega_m + \Omega_{\Lambda} = 1 \Rightarrow \Omega_{\Lambda} = 1 - \Omega_m$

- Constraint on current drift of $\alpha$ by Rosenband:

$\left( \frac{1}{H}\frac{\dot{\alpha}}{\alpha} \right)_0 = - 0.22 \pm 0.32 ppm$

In [6]:
#Constraint Rosenband
#unc_Rosenband = .32*10**(-6)
#unc_Rosenband_F = .032*10**(-6) #Future constraint = unc_Rosenvand/10

#Constraint Rosenband in ppm
unc_Rosenband = .32
unc_Rosenband_F = .032 #Future constraint = unc_Rosenvand/10


#Considering OmegaM as constant
a = -0.13
b = -1.74

FpriorClocks_Oconst  = np.array([[b**2*unc_Rosenband**(-2), a*b*unc_Rosenband**(-2)], [a*b*unc_Rosenband**-2, a**2*unc_Rosenband**(-2)]])
FpriorClocksF_Oconst = np.array([[b**2*unc_Rosenband_F**(-2), a*b*unc_Rosenband_F**(-2)], [a*b*unc_Rosenband_F**-2, a**2*unc_Rosenband_F**(-2)]])


#Considering OmegaM as parameter
d = 2 * ( (OmegaM / np.sqrt(1 - OmegaM)) * np.log((1+np.sqrt(1 - OmegaM))/np.sqrt(OmegaM)) ) - 1
e = -4 * (OmegaM / np.sqrt(1 - OmegaM)) * np.log((1+np.sqrt(1 - OmegaM))/np.sqrt(OmegaM))

FpriorClocks_Oparam  = np.array([[e**2*unc_Rosenband**(-2), d*e*unc_Rosenband**(-2)], [d*e*unc_Rosenband**(-2), d**2*unc_Rosenband**(-2)]])
FpriorClocksF_Oparam = np.array([[e**2*unc_Rosenband_F**(-2), d*e*unc_Rosenband_F**(-2)], [d*e*unc_Rosenband_F**(-2), d**2*unc_Rosenband_F**(-2)]])

In [7]:
def addPriors_BEK(F_ab, cur_ab, OCP):
    #Assuming OmegaM as a constant
    if OCP == 'constant':
        newF_ab = np.copy(F_ab)
        if cur_ab == 1:
            newF_ab = F_ab + FpriorClocks_Oconst
        else:
            newF_ab = F_ab + FpriorClocksF_Oconst 
    #Assuming OmegaM as a parameter
    if OCP == 'parameter':
        newF_ab = np.copy(F_ab)
        if cur_ab == 1:
            FpriorClocksNew = np.insert(FpriorClocks_Oparam, 0, 0, axis=0)
            FpriorClocksNew = np.insert(FpriorClocksNew, 0, 0, axis=1)
            newF_ab = F_ab + FpriorClocksNew
        else:
            FpriorClocksFNew = np.insert(FpriorClocksF_Oparam, 0, 0, axis=0)
            FpriorClocksFNew = np.insert(FpriorClocksFNew, 0, 0, axis=1)
            newF_ab = F_ab + FpriorClocksFNew
  
    return newF_ab

In [8]:
def add1DPriors(F, priors): #to add priors locally in the intended parameters
    '''F-Fisher matrix, priors-[[sigma_prior1,local1],...]'''
    G = np.copy(F)
    for i in range(len(priors)): 
        G[priors[i][1],priors[i][1]]+=priors[i][0]**-2
    return G

# Measurements
### - Redshifts

In [9]:
#ANDES Baseline
zAndes_15baseline = np.linspace(1,2.5,num=15)
zAndes_10baseline = np.linspace(2.5,4,num=10)


#ANDES Optimistic
zAndes_20optimistic = np.linspace(1,2.5,num=20)
zAndes_15optimistic1 = np.linspace(2.5,4,num=15)
zAndes_15optimistic2 = np.linspace(4,7,num=15)

### - Uncertainties

In [10]:
#ANDES Baseline

#uncAndes_15baseline = np.zeros(15) + (0.1*10**(-6))
#uncAndes_10baseline = np.zeros(10) + (0.2*10**(-6))

#PPM:
uncAndes_15baseline = np.zeros(15) + (0.1)
uncAndes_10baseline = np.zeros(10) + (0.2)


#ANDES Optimistic

#uncAndes_20optimistic = np.zeros(20) + (0.05*10**(-6))
#uncAndes_15optimistic1 = np.zeros(15) + (0.1*10**(-6))
#uncAndes_15optimistic2 = np.zeros(15) + (1*10**(-6))

#PPM:
uncAndes_20optimistic = np.zeros(20) + (0.05)
uncAndes_15optimistic1 = np.zeros(15) + (0.1)
uncAndes_15optimistic2 = np.zeros(15) + (1)

# Fisher Matrices
## - No Priors
#### Bekenstein $(\zeta_\Lambda = \zeta_m = 0)$

In [11]:
FM_BEK0_Andes_15baseline    = FisherMatrix_ab(zAndes_15baseline, uncAndes_15baseline, OmegaM, w0=None, wa=None, Zl=0, Zm=0, OCP= 'parameter', analysis = 'alpha')
FM_BEK0_Andes_10baseline    = FisherMatrix_ab(zAndes_10baseline, uncAndes_10baseline, OmegaM, w0=None, wa=None, Zl=0, Zm=0, OCP= 'parameter', analysis = 'alpha')

FM_BEK0_Andes_20optimistic  = FisherMatrix_ab(zAndes_20optimistic, uncAndes_20optimistic, OmegaM, w0=None, wa=None, Zl=0, Zm=0, OCP= 'parameter', analysis = 'alpha')
FM_BEK0_Andes_15optimistic1 = FisherMatrix_ab(zAndes_15optimistic1, uncAndes_15optimistic1, OmegaM, w0=None, wa=None, Zl=0, Zm=0, OCP= 'parameter', analysis = 'alpha')
FM_BEK0_Andes_15optimistic2 = FisherMatrix_ab(zAndes_15optimistic2, uncAndes_15optimistic2, OmegaM, w0=None, wa=None, Zl=0, Zm=0, OCP= 'parameter', analysis = 'alpha')


BEK0_AlphaB = FM_BEK0_Andes_baseline = FM_BEK0_Andes_15baseline + FM_BEK0_Andes_10baseline
BEK0_AlphaO = FM_BEK0_Andes_optimistic = FM_BEK0_Andes_20optimistic + FM_BEK0_Andes_15optimistic1 + FM_BEK0_Andes_15optimistic2

#### Bekenstein $(\zeta_\Lambda = \zeta_m = -0.1)$

In [12]:
FM_BEK_Andes_15baseline    = FisherMatrix_ab(zAndes_15baseline, uncAndes_15baseline, OmegaM, w0=None, wa=None, Zl=-0.1, Zm=-0.1, OCP= 'parameter', analysis = 'alpha')
FM_BEK_Andes_10baseline    = FisherMatrix_ab(zAndes_10baseline, uncAndes_10baseline, OmegaM, w0=None, wa=None, Zl=-0.1, Zm=-0.1, OCP= 'parameter', analysis = 'alpha')

FM_BEK_Andes_20optimistic  = FisherMatrix_ab(zAndes_20optimistic, uncAndes_20optimistic, OmegaM, w0=None, wa=None, Zl=-0.1, Zm=-0.1, OCP= 'parameter', analysis = 'alpha')
FM_BEK_Andes_15optimistic1 = FisherMatrix_ab(zAndes_15optimistic1, uncAndes_15optimistic1, OmegaM, w0=None, wa=None, Zl=-0.1, Zm=-0.1, OCP= 'parameter', analysis = 'alpha')
FM_BEK_Andes_15optimistic2 = FisherMatrix_ab(zAndes_15optimistic2, uncAndes_15optimistic2, OmegaM, w0=None, wa=None, Zl=-0.1, Zm=-0.1, OCP= 'parameter', analysis = 'alpha')


FM_BEK_Andes_baseline = FM_BEK_Andes_15baseline + FM_BEK_Andes_10baseline
FM_BEK_Andes_optimistic = FM_BEK_Andes_20optimistic + FM_BEK_Andes_15optimistic1 + FM_BEK_Andes_15optimistic2

#### CPL $(w_0 = -1)$ and $(w_a = 0)$

In [13]:
FM_CPLLCDM_Andes_15baseline = FisherMatrix_aCPL(zAndes_15baseline, uncAndes_15baseline, OmegaM=0.3, w0=-1, wa=0, zeta=-0.1, analysis = 'alpha')
FM_CPLLCDM_Andes_10baseline = FisherMatrix_aCPL(zAndes_10baseline, uncAndes_10baseline, OmegaM=0.3, w0=-1, wa=0, zeta=-0.1, analysis = 'alpha')

FM_CPLLCDM_Andes_20optimistic = FisherMatrix_aCPL(zAndes_20optimistic, uncAndes_20optimistic, OmegaM=0.3, w0=-1, wa=0, zeta=-0.1, analysis = 'alpha')
FM_CPLLCDM_Andes_15optimistic1 = FisherMatrix_aCPL(zAndes_15optimistic1, uncAndes_15optimistic1, OmegaM=0.3, w0=-1, wa=0, zeta=-0.1, analysis = 'alpha')
FM_CPLLCDM_Andes_15optimistic2 = FisherMatrix_aCPL(zAndes_15optimistic2, uncAndes_15optimistic2, OmegaM=0.3, w0=-1, wa=0, zeta=-0.1, analysis = 'alpha')


FM_CPLLCDM_Andes_baseline = FM_CPLLCDM_Andes_15baseline + FM_CPLLCDM_Andes_10baseline
FM_CPLLCDM_Andes_optimistic = FM_CPLLCDM_Andes_20optimistic + FM_CPLLCDM_Andes_15optimistic1 + FM_CPLLCDM_Andes_15optimistic2

#### CPL $(w_0 = -0.9)$ and $(w_a = 0.3)$

In [14]:
FM_CPL_Andes_15baseline = FisherMatrix_aCPL(zAndes_15baseline, uncAndes_15baseline, OmegaM=0.3, w0=-0.9, wa=0.3, zeta=-0.1, analysis = 'alpha')
FM_CPL_Andes_10baseline = FisherMatrix_aCPL(zAndes_10baseline, uncAndes_10baseline, OmegaM=0.3, w0=-0.9, wa=0.3, zeta=-0.1, analysis = 'alpha')

FM_CPL_Andes_20optimistic = FisherMatrix_aCPL(zAndes_20optimistic, uncAndes_20optimistic, OmegaM=0.3, w0=-0.9, wa=0.3, zeta=-0.1, analysis = 'alpha')
FM_CPL_Andes_15optimistic1 = FisherMatrix_aCPL(zAndes_15optimistic1, uncAndes_15optimistic1, OmegaM=0.3, w0=-0.9, wa=0.3, zeta=-0.1, analysis = 'alpha')
FM_CPL_Andes_15optimistic2 = FisherMatrix_aCPL(zAndes_15optimistic2, uncAndes_15optimistic2, OmegaM=0.3, w0=-0.9, wa=0.3, zeta=-0.1, analysis = 'alpha')


FM_CPL_Andes_baseline = FM_CPL_Andes_15baseline + FM_CPL_Andes_10baseline
FM_CPL_Andes_optimistic = FM_CPL_Andes_20optimistic + FM_CPL_Andes_15optimistic1 + FM_CPL_Andes_15optimistic2

## - With Priors
#### Bekenstein $(\zeta_\Lambda = \zeta_m = 0)$

It has no Priors because $\zeta_\Lambda = \zeta_m = 0$ implies LambdaCDM which itself implies $\frac{\Delta \alpha}{\alpha} = 0$

#### Bekenstein $(\zeta_\Lambda = \zeta_m = -0.1)$

In [15]:
#Considering the prior sigma(OmegaM)=0.03=uncOmegaM and present atomic clocks (cur_ab=1)

CurrentClocksbaseline = add1DPriors(FM_BEK_Andes_baseline, [[uncOmegaM,0]])
BEK_AlphaB = addPriors_BEK(CurrentClocksbaseline  , cur_ab=1, OCP= 'parameter')

CurrentClocksoptimistic = add1DPriors(FM_BEK_Andes_optimistic, [[uncOmegaM,0]])
BEK_AlphaO = addPriors_BEK(CurrentClocksoptimistic  , cur_ab=1, OCP= 'parameter')

#### CPL $(w_0 = -1)$ and $(w_a = 0)$

In [16]:
CPLLCDM_AlphaB = add1DPriors(FM_CPLLCDM_Andes_baseline, [[uncOmegaM,0], [unczeta,1]])
CPLLCDM_AlphaO = add1DPriors(FM_CPLLCDM_Andes_optimistic, [[uncOmegaM,0], [unczeta,1]])

#### CPL $(w_0 = -0.9)$ and $(w_a = 0.3)$

In [17]:
CPL_AlphaB = add1DPriors(FM_CPL_Andes_baseline, [[uncOmegaM,0],[uncw0Cur,1], [uncwaCur,2], [unczeta,3]])
CPL_AlphaO = add1DPriors(FM_CPL_Andes_optimistic, [[uncOmegaM,0],[uncw0Cur,1], [uncwaCur,2], [unczeta,3]])

# Confidence ellipses (1-sigma)
#### Bekenstein

In [18]:
def grafZmZl_BEK(Zm,Zl,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'OmegaM',analysis='alpha-BEK')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle2(Fs[i])
        ells.append([Zm,Zl,Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$\zeta_m$ (ppm)','$\zeta_\Lambda$ (ppm)'])
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.xlim(-0.25,0.05)
    plt.ylim(-1,0.8)
    #plt.savefig('BEKZmZl.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#grafZmZl_BEK(-0.1,-0.1,[BEK_AlphaB, BEK_AlphaO])

In [19]:
def grafOmZm_BEK(OmegaM,Zm,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'Zl',analysis='alpha-BEK')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle2(Fs[i])
        ells.append([OmegaM,Zm,Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$\Omega_m$','$\zeta_m$ (ppm)'])
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.xlim(0.24,0.36)
    plt.ylim(-0.25,0.05)
    #plt.savefig('BEKOmegaMZm.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#grafOmZm_BEK(0.3,-0.1,[BEK_AlphaB, BEK_AlphaO])

In [20]:
def grafOmZl_BEK(OmegaM,Zl,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'Zm',analysis='alpha-BEK')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle2(Fs[i])
        ells.append([OmegaM,Zl,Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$\Omega_m$','$\zeta_\Lambda$ (ppm)'])
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.xlim(0.24,0.36)
    plt.ylim(-1,0.8)
    #plt.savefig('BEKOmegaMZl.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#grafOmZl_BEK(0.3,-0.1,[BEK_AlphaB, BEK_AlphaO])

#### CPL

In [21]:
def grafwazeta_CPL(wa,zetas,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'w0',analysis='alpha-CPL')
        Fm = marginalize(Fm,'OmegaM',analysis='alpha-CPL')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle2(Fs[i])
        ells.append([wa,zetas, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$w_a$','$\zeta$ (ppm)'])
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.xlim(-0.4,1)
    plt.ylim(-0.25, 0.05)
    #plt.savefig('CPLAlphawazeta.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#grafwazeta_CPL(0.3,-0.1,[CPL_AlphaB, CPL_AlphaO])

In [22]:
def grafw0zeta_CPL(w0,zetas,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'wa',analysis='alpha-CPL')
        Fm = marginalize(Fm,'OmegaM',analysis='alpha-CPL')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle2(Fs[i])
        ells.append([w0,zetas, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$w_0$','$\zeta$ (ppm)'])
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.xlim(-1.2,-0.6)
    plt.ylim(-0.25, 0.05)
    #plt.savefig('CPLAlphaw0zeta.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#grafw0zeta_CPL(-0.9,-0.1,[CPL_AlphaB, CPL_AlphaO])

In [23]:
def grafOmegaMzeta_CPL(OmegaM,zetas,Fs):
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'wa',analysis='alpha-CPL')
        Fm = marginalize(Fm,'w0',analysis='alpha-CPL')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle2(Fs[i])
        ells.append([OmegaM,zetas, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$\Omega_m$','$\zeta$ (ppm)'])
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.xlim(0.24,0.36)
    plt.ylim(-0.25, 0.05)
    #plt.savefig('CPLAlphaOmegaMzeta.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#grafOmegaMzeta_CPL(0.3,-0.1,[CPL_AlphaB, CPL_AlphaO])

In [24]:
def grafw0wa_CPL(w0,wa,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'zeta',analysis='alpha-CPL')
        Fm = marginalize(Fm,'OmegaM',analysis='alpha-CPL')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle2(Fs[i])
        ells.append([w0,wa, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$w_0$','$w_a$'])
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.xlim(-1.2,-0.6)
    plt.ylim(-0.4,1)
    #plt.savefig('CPLw0wa.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#grafw0wa_CPL(-0.9,0.3,[CPL_AlphaB, CPL_AlphaO])

In [25]:
def grafOmegaMw0_CPL(OmegaM,w0,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'zeta',analysis='alpha-CPL')
        Fm = marginalize(Fm,'wa',analysis='alpha-CPL')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle2(Fs[i])
        ells.append([OmegaM,w0, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$\Omega_m$','$w_0$'])
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.xlim(0.24,0.36)
    plt.ylim(-1.2,-0.6)
    #plt.savefig('CPLOmegaMw0.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#grafOmegaMw0_CPL(0.3,-0.9,[CPL_AlphaB, CPL_AlphaO])

In [26]:
def grafOmegaMwa_CPL(OmegaM,wa,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'zeta',analysis='alpha-CPL')
        Fm = marginalize(Fm,'w0',analysis='alpha-CPL')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle2(Fs[i])
        ells.append([OmegaM,wa, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$\Omega_m$','$w_a$'])
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.xlim(0.24,0.36)
    plt.ylim(-0.4,1)
    #plt.savefig('CPLOmegaMwa.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#grafOmegaMwa_CPL(0.3,0.3,[CPL_AlphaB, CPL_AlphaO])

# Tables
#### Bekenstein

In [27]:
def tableBEKalpha(matrices):
    i=0
    df=pd.DataFrame()
    nformat = "{:.0f}"
    float_format="{:.3f}"
    zetaformat="{:.3e}"
    

    for matrix in matrices:
        rows = {'ρ(Ωm, Zm)':float_format.format(FigMeriteRho(matrix,['Zl'],analysis='alpha-BEK')[1]),
                'ρ(Ωm, Zl)':float_format.format(FigMeriteRho(matrix,['Zm'],analysis='alpha-BEK')[1]),
                'ρ(Zm, Zl)':float_format.format(FigMeriteRho(matrix,['OmegaM'],analysis='alpha-BEK')[1]),                      
                'FoM(Ωm, Zm)': nformat.format(FigMeriteRho(matrix,['Zl'],analysis='alpha-BEK')[0]),
                'FoM(Ωm, Zl)': nformat.format(FigMeriteRho(matrix,['Zm'],analysis='alpha-BEK')[0]),
                'FoM(Zm, Zl)': nformat.format(FigMeriteRho(matrix,['OmegaM'],analysis='alpha-BEK')[0]),
                'σ(Ωm)':float_format.format(Sigmas(matrix)[0]),
                'σ(Zm)':float_format.format(Sigmas(matrix)[1]),
                'σ(Zl)':float_format.format(Sigmas(matrix)[2])}
        i+=1
        df = df.append(rows,ignore_index=True)
        del rows
    
    
    Datasets = ['Baseline','Optimistic']
    df.insert(0, "Parameters", Datasets, True)
    
    df = df.transpose()
    
            
    return df

In [28]:
#Bekenstein - Baseline + Optimistic

#matricesBEKalpha = [BEK_AlphaB, BEK_AlphaO]
#tab1 = tableBEKalpha(matricesBEKalpha)

#See the table in DataFrame:
#tab1

#Transform dataframe to latex code:
#print(tab1.to_latex(header=False,index=True))

#### CPL

In [29]:
def tableCPLalpha(matrices):
    i=0
    df=pd.DataFrame()
    nformat = "{:.0f}"
    float_format="{:.3f}"
    zetaformat="{:.3e}"
    

    for matrix in matrices:
        rows = {'ρ(Ωm, w0)':float_format.format(FigMeriteRho(matrix,['wa','zeta'],analysis='alpha-CPL')[1]),
                'ρ(Ωm, wa)':float_format.format(FigMeriteRho(matrix,['w0','zeta'],analysis='alpha-CPL')[1]),
                'ρ(Ωm, zeta)':float_format.format(FigMeriteRho(matrix,['w0','wa'],analysis='alpha-CPL')[1]),                       
                'ρ(w0, wa)':float_format.format(FigMeriteRho(matrix,['OmegaM','zeta'],analysis='alpha-CPL')[1]),
                'ρ(w0, zeta)':float_format.format(FigMeriteRho(matrix,['OmegaM','wa'],analysis='alpha-CPL')[1]),
                'ρ(wa, zeta)':float_format.format(FigMeriteRho(matrix,['OmegaM','w0'],analysis='alpha-CPL')[1]),
                'FoM(Ωm, w0)': nformat.format(FigMeriteRho(matrix,['wa','zeta'],analysis='alpha-CPL')[0]),
                'FoM(Ωm, wa)': nformat.format(FigMeriteRho(matrix,['w0','zeta'],analysis='alpha-CPL')[0]),
                'FoM(Ωm, zeta)': nformat.format(FigMeriteRho(matrix,['w0','wa'],analysis='alpha-CPL')[0]),
                'FoM(w0, wa)': nformat.format(FigMeriteRho(matrix,['OmegaM','zeta'],analysis='alpha-CPL')[0]),
                'FoM(w0, zeta)': nformat.format(FigMeriteRho(matrix,['OmegaM','wa'],analysis='alpha-CPL')[0]),
                'FoM(wa, zeta)': nformat.format(FigMeriteRho(matrix,['OmegaM','w0'],analysis='alpha-CPL')[0]),
                'σ(Ωm)':float_format.format(Sigmas(matrix)[0]),
                'σ(w0)':float_format.format(Sigmas(matrix)[1]),
                'σ(wa)':float_format.format(Sigmas(matrix)[2]),
                'σ(zeta)':float_format.format(Sigmas(matrix)[3])}
        i+=1
        df = df.append(rows,ignore_index=True)
        del rows
    
    
    Datasets = ['Baseline','Optimistic']
    df.insert(0, "Parameters", Datasets, True)
    
    df = df.transpose()
    
            
    return df

In [30]:
#CPL - Baseline + Optimistic

#matricesCPLalpha = [CPL_AlphaB, CPL_AlphaO]
#tab1 = tableCPLalpha(matricesCPLalpha)

#See the table in DataFrame:
#tab1

#Transform dataframe to latex code:
#print(tab1.to_latex(header=False,index=True))